#Glaucoma Identification


# Install Libraries and Datasets

In [ ]:
# Installing libraries

import tensorflow as tf
import random, os
import shutil
from matplotlib.image import imread
import sklearn
from sklearn.model_selection import train_test_split
import os
import random
import numpy as np
import pandas as pd
import cv2
import imageio
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.applications.mobilenet import MobileNet
from keras.metrics import AUC

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# Installing the glaucoma data set

!wget https://www.dropbox.com/sh/fk7qcyg1zzny19a/AADb0VsFaseMExvmdMgy2T7Fa?dl=0

In [ ]:
#Unzip the dataset

!unzip AADb0VsFaseMExvmdMgy2T7Fa?dl=0

# Data Management & Exploration

In [ ]:
import os
import numpy as np
from PIL import Image

# location of images
path = '/content/Images'

# initialize empty lists for storing the images
no_glaucoma_images = []
glaucoma_images = []

# loop through all the images in the folder
for image_name in os.listdir(path):
    # read image
    image = Image.open(os.path.join(path, image_name))
    # convert image to numpy array
    image_array = np.array(image)
    # check if the image name contains 'g'
    if 'g_ACRIMA' in image_name:
        # store the image in glaucoma_images list
        glaucoma_images.append(image_array)
    else:
        # store the image in no_glaucoma_images list
        no_glaucoma_images.append(image_array)

# convert the lists to numpy arrays
no_glaucoma_images = np.array(no_glaucoma_images)
glaucoma_images = np.array(glaucoma_images)

In [ ]:
# Combine the arrays
combined_array = np.concatenate([no_glaucoma_images, glaucoma_images])

# Create a label array with the binary labels
zeroes = np.zeros(len(no_glaucoma_images))
ones = np.ones(len(glaucoma_images))
label_array = np.concatenate([zeroes, ones])

In [ ]:
def resize_image(image, size=(224, 224)):
  # Use cv2.resize() to resize the image
  resized_image = cv2.resize(image, size)

  return resized_image

# Resize the images in the combined_array array
resized_combined_array = []
for image in combined_array:
  resized_image = resize_image(image)
  resized_combined_array.append(resized_image)

# Convert the list to a NumPy array
resized_combined_array = np.array(resized_combined_array)

plt.imshow(resized_combined_array[0])
plt.show()

In [ ]:
# Verifying that all images are the same size -- 224 x 224

same_size = True
reference_shape = resized_combined_array[0].shape
for i in range(1, len(combined_array)):
    if resized_combined_array[i].shape != reference_shape:
        same_size = False
        break

if same_size:
    print("All images are the same size")

    height, width = resized_combined_array.shape[:2]
    print("Image size: {} x {}".format(height, width))
else:
    print("Not all images are the same size")

In [ ]:
# Printing out a few image examples

plt.imshow(resized_combined_array[0])
plt.show()

plt.imshow(resized_combined_array[200])
plt.show()

print(len(resized_combined_array))

print(label_array[5])
print(label_array[400])

In [ ]:
# Download Some Images For Testing

number = 50

cv2.imwrite('no1.jpg', resized_combined_array[number])

In [ ]:
# Matplotlib Bar Graph Construction -- Verify Dataset

# Count the number of no_dr0 and yes_dr1 images in the combined_array
num_positive = glaucoma_images.shape[0]
num_negative = no_glaucoma_images.shape[0]

# Create a figure and axis
fig, ax = plt.subplots()

# Create a bar plot with two bars, one for no_dr0 and one for yes_dr1
ax.bar(0, num_positive, label="num_positive")
ax.bar(1, num_negative, label="num_negative")

# Set the x-axis tick labels
ax.set_xticks([0, 1])
ax.set_xticklabels(["positive", "negative"])

# Set the y-axis tick labels
ax.set_ylabel("Number of images")

# Add labels to the bar graphs
ax.text(0, num_positive + 50, str(num_positive), ha="center", va="bottom", fontsize=14, color="red", weight="bold")
ax.text(1, num_negative + 50, str(num_negative), ha="center", va="bottom", fontsize=14, color="red", weight="bold")

# Add a legend
ax.legend()

# Show the plot
plt.show()

#Machine Learning Algorithm


In [ ]:
# Necessary imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Extract the features and labels from the combined_array_2D array:
X = resized_combined_array # features (colored images)
y = label_array # labels (diagnosis of DR)

# Reshape the features to have 4 dimensions (batch_size, height, width, channels)
X = X.reshape((X.shape[0], X.shape[1], X.shape[2], 3))

# Split the dataset into training, test, and validation sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25 / 0.75, random_state=42)

# Create the data generator for data augmentation
datagen = ImageDataGenerator (
  rescale=1./255,
  rotation_range=45,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest'
)

# Create the CNN model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), kernel_initializer='he_uniform', activation='relu', input_shape=(X.shape[1], X.shape[2], 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), kernel_initializer='he_uniform', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3), kernel_initializer='he_uniform', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the augmented data
history = model.fit(
  datagen.flow(X_train, y_train, batch_size=32),
  epochs=30,
  validation_data=(X_val, y_val),
  verbose=1
)

# Evaluate the model on the test data
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Confusion Matrix

- True Positives, True Negatives, False Positives, False Negatives

In [ ]:
from sklearn.metrics import classification_report

# Predict the labels for the test data
y_pred = model.predict(X_test)

# Convert the predicted probabilities to binary labels
y_pred = (y_pred > 0.5).astype(int)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Use the 'Blues' colormap
plt.imshow(cm, cmap='Blues')

# Define the labels for the cells
labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']

# Add labels to the cells
for i in range(2):
  for j in range(2):
    plt.text(j, i, labels[2 * i + j], ha='center', va='center', color='black')

# Add the actual numbers to the cells
for i in range(2):
  for j in range(2):
    plt.text(j, i + 0.2, cm[i, j], ha='center', va='center', color='black')

# Adjust the axis labels and title
plt.colorbar()
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('True vs. Predicted Detection for Glaucoma')

# Adjust the figure size
plt.figure(figsize=(8, 8))

# Display the plot
plt.show()

# Generate a classification report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Positive'])
print(report)

# ROC Curve

In [ ]:
# ROC
from sklearn.metrics import roc_curve
y_preds = model.predict(X_test).ravel()

fpr, tpr, thresholds = roc_curve(y_test, y_preds)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'y--')
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC Curve for Glaucoma')
plt.show()

from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_preds)
print(auc)

In [ ]:
# Learning Curve and Loss Curve:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

# TensorFlow Lite Implementation -- Android App


In [ ]:
# Convert Model to TensorFlow Lite -- App Usage -- DO NOT RUN THIS EVERYTIME

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to file
with open('model2.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Test the TensorFlow Lite Model

img = cv2.imread("no1.jpg")
img = cv2.resize(img, (224,224))
img = np.array(img, dtype="float32")
img = np.reshape(img, (1,224,224,3))

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model2.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']

print(input_details)
print(output_details)

interpreter.set_tensor(input_details[0]['index'], img)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)